## Install and Import Libraries

In [32]:
## Install and Import Libraries
!pip install openai
!pip install openai-agents
!pip install rich
import asyncio

import textwrap
from openai import OpenAI
from agents import Agents, Runner

ImportError: cannot import name 'Agents' from 'agents' (/Users/jamesgray/anaconda3/envs/openai/lib/python3.13/site-packages/agents/__init__.py)

## Responses API with Web Search

In [21]:
client = OpenAI()

def webSearch (query: str) -> str:
    response = client.responses.create(
        model="gpt-4o",
        tools=[{"type": "web_search_preview"}],
        input=query
    )
    return response.output_text

raw_response = webSearch("What was a positive news story from today?")
wrapped_response = textwrap.fill(raw_response, width=120)
print(wrapped_response)

Here are some positive news stories from today, March 16, 2025:  **Gaza Ceasefire Agreement**  Israel and Hamas have
agreed to a ceasefire and hostage release deal, set to commence today. This agreement aims to halt a conflict that has
resulted in over 46,000 Palestinian casualties over the past 15 months. Human rights agencies have cautiously welcomed
the deal, urging all parties to adhere to its terms. ([positive.news](https://www.positive.news/society/good-news-
stories-from-week-03-of-2025/?utm_source=openai))  **Indonesia's Free Meals Initiative**  Indonesia has launched an
ambitious program to provide free meals to more than a quarter of its population. The initiative targets 82.9 million
people by 2029, focusing on schoolchildren and pregnant women. Currently, 190 kitchens have been established across the
country to support this effort. ([positive.news](https://www.positive.news/society/good-news-stories-from-
week-03-of-2025/?utm_source=openai))  **SpaceX Astronaut Rescue Missi

## Responses API with File Search


In [23]:
import json
from rich import print_json
client = OpenAI()

response = client.responses.create(
    model="gpt-4o-mini",
    input="How can I manage my self-limiting beliefs?",
    tools=[{
        "type": "file_search",
        "vector_store_ids": ["vs_67d7245bd5a48191b1a81ccabca975e9"]
    }],
    include=["file_search_call.results"]
)
# Convert response to JSON to view the results
response_json = response.to_json()
print(response_json)
# Print the output text
print (response.output_text)


{
  "id": "resp_67d8b94747188192ad5d30a2cea4d19f00863aa571685be9",
  "created_at": 1742256455.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-4o-mini-2024-07-18",
  "object": "response",
  "output": [
    {
      "id": "fs_67d8b948130c81929a090ae6d877c55a00863aa571685be9",
      "queries": [
        "How can I manage my self-limiting beliefs?",
        "Strategies for overcoming self-limiting beliefs",
        "Managing self-limiting beliefs techniques"
      ],
      "status": "completed",
      "type": "file_search_call",
      "results": [
        {
          "attributes": {},
          "file_id": "file-An7GonK71ajjq2mXteSE7E",
          "filename": "lesson 1.2.4 - how to reframe self-limiting beliefs.docx",
          "score": 0.9275570151141759,
          "text": "INNER QUEST by James Gray\n\n\n\nHow to Reframe Self-Limiting Beliefs\n\nReframing Process\n\nHere is a stepwise process to help you learn how to reframe self-li

## Agents SDK with Custom Functions

In [43]:
from agents import Agent, Runner, function_tool

@function_tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny."

agent = Agent(
    name="Weather Agent",
    instructions="You are a helpful agent that can answer questions about the weather forecast.",
    tools=[get_weather],
    model="gpt-4o-mini"
)

result = await Runner.run(agent, input="What's the current weather in Austin, TX?")
print(result.final_output)


The current weather in Austin, TX is sunny.


## Workflow Chain with Agents SDK
This is an example of a deterministic workflow of 3 steps, where each step is performed by an agent.
1. The first agent is a research agent that scans the web for the top 5 AI stories in the last 7 days.
2. The second agent summarizes the story into 200-300 characters and creates a title.
3. The third agent publishes the story as markdown.

In [45]:
from agents import Agent, Runner, trace, WebSearchTool

# Define the agents
research_agent = Agent(
    name="Research Agent",
    instructions="You are a research agent that scans the web for the top 5 AI stories in the last 7 days.",
    model="gpt-4o-mini",    
    tools=[WebSearchTool()]
)

writer_agent = Agent(
    name="Writer Agent",
    instructions="You are a writer agent that summarizes the story into 200-300 characters and creates a title.",
    model="gpt-4o-mini"
)

publisher_agent = Agent(
    name="Publisher Agent",
    instructions="You are a publisher agent that publishes the story as markdown.",
    model="gpt-4o-mini"
)

# Chain the agents
async def chain_agents():
    # Run the research agent
    research_result = await Runner.run(research_agent, input="What are the top 5 AI stories in the last 7 days?")
    print(research_result.final_output)

    # Run the writer agent
    writer_result = await Runner.run(writer_agent, input=research_result.final_output)
    print(writer_result.final_output)

    # Run the publisher agent
    publisher_result = await Runner.run(publisher_agent, input=writer_result.final_output)
    print(publisher_result.final_output)

# For direct execution in Jupyter notebook:
result = await chain_agents()
    

Here are the top five AI stories from the past week:

1. **Alibaba's Strategic Shift Towards AI**

   Jack Ma's return to Alibaba has marked a significant transformation for the company. Inspired by OpenAI's ChatGPT, Alibaba has heavily invested in artificial intelligence, developing its own large language models, Qwen. This strategic pivot has revitalized the company, leading to a 66% increase in share price at the start of 2025. Despite facing competition from domestic rivals and geopolitical challenges, Alibaba's AI-focused roadmap demonstrates resilience and confidence. ([ft.com](https://www.ft.com/content/df2bccee-1730-402f-bb92-9d743018324f?utm_source=openai))

2. **Alden Newspapers Oppose AI Copyright Proposals**

   Dozens of newspapers owned by Alden Global Capital have published editorials criticizing OpenAI and Google for seeking looser AI copyright restrictions. Both companies have requested the legality of training AI models on public data, including copyrighted material. 